In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam

from keras.callbacks import EarlyStopping


AttributeError: partially initialized module 'tensorflow.python.eager.def_function' has no attribute 'function' (most likely due to a circular import)

In [ ]:
X_train_all = []
y_train_all = []

train_files = ['/kaggle/input/test-anomaly/0.csv',
               '/kaggle/input/test-anomaly//2.csv',
               '/kaggle/input/test-anomaly/3.csv',
               '/kaggle/input/test-anomaly/4.csv',
               '/kaggle/input/test-anomaly/5.csv',
               
               '/kaggle/input/train-normal/0.csv',
               '/kaggle/input/train-normal/1.csv',
               '/kaggle/input/train-normal/2.csv',
               '/kaggle/input/train-normal/3.csv',
               '/kaggle/input/train-normal/4.csv',
               '/kaggle/input/train-normal/5.csv',
               '/kaggle/input/train-normal/6.csv',
               '/kaggle/input/train-normal/7.csv',
               '/kaggle/input/train-normal/8.csv']
for file in train_files:
    train_df = pd.read_csv(file)
    label_encoder = LabelEncoder()
    train_df['type_encoded'] = label_encoder.fit_transform(train_df['type'])
    X_train_all.append(train_df[['time_pos', 'type_encoded', 'byte_pos', 'size']].values)
    y_train_all.append(train_df[['is_normal', 'has_move', 'has_blur', 'has_block', 'has_overexposure']].values)

X_train = np.concatenate(X_train_all)
y_train = np.concatenate(y_train_all)


In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax')) 

In [ ]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
test_files = ['/kaggle/input/meta-test/1.mp4.csv',
              '/kaggle/input/meta-test/2.mp4.csv',
              '/kaggle/input/meta-test/3.mp4.csv',
              '/kaggle/input/meta-test/4.mp4.csv',
              '/kaggle/input/meta-test/5.mp4.csv'] 
for file in test_files:
    test_df = pd.read_csv(file)
    test_df['type_encoded'] = label_encoder.transform(test_df['type'])
    X_test = test_df[['time_pos', 'type_encoded', 'byte_pos', 'size']].values
    y_test = test_df[['is_normal', 'has_move', 'has_blur', 'has_block', 'has_overexposure']].values
    
    X_train_reshaped = X_train.reshape((1, X_train.shape[0], X_train.shape[1]))
    X_test_reshaped = X_test.reshape((1, X_test.shape[0], X_test.shape[1]))

    history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", accuracy)